In [1]:
!nvidia-smi

Wed Mar 12 04:20:49 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla P100-PCIE-16GB           Off |   00000000:00:04.0 Off |                    0 |
| N/A   35C    P0             24W /  250W |       0MiB /  16384MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
!pip install gdown

In [4]:
! rm -rf video.mp4

In [5]:
import gdown

# Update with your file's specific ID
file_id = "1NNhyO_PUrfy3dVat_BmlYaBSDCLIzo2Q"
url = f"https://drive.google.com/uc?id={file_id}"

output = "vehicle-counting.mp4"
gdown.download(url, output, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1NNhyO_PUrfy3dVat_BmlYaBSDCLIzo2Q
To: /kaggle/working/vehicle-counting.mp4
100%|██████████| 35.3M/35.3M [00:00<00:00, 67.8MB/s]


'vehicle-counting.mp4'

In [6]:
import os
HOME = os.getcwd()
print(HOME)

/kaggle/working


In [38]:
SOURCE_VIDEO_PATH = "/kaggle/working/vehicle-counting.mp4"

In [8]:
# Pip install method (recommended)

!pip install "ultralytics<=8.3.40"

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics 8.3.40 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
Setup complete ✅ (4 CPUs, 31.4 GB RAM, 6170.1/8062.4 GB disk)


In [14]:
!pip install supervision==0.3.0

from IPython import display
display.clear_output()

import supervision
print("supervision.__version__:", supervision.__version__)

supervision.__version__: 0.3.0


In [9]:
# settings
MODEL = "yolov8x.pt"

In [10]:
from ultralytics import YOLO

model = YOLO(MODEL)
model.fuse()

100%|██████████| 131M/131M [00:00<00:00, 253MB/s] 


YOLOv8x summary (fused): 268 layers, 68,200,608 parameters, 0 gradients, 257.8 GFLOPs


## Track and count

In [ ]:
import supervision as sv
from ultralytics import YOLO
import os
import json
import cv2  # OpenCV for image saving
import numpy as np


TARGET_VIDEO_PATH = 'output_video.mp4'
FRAME_SAVE_DIR = 'frames/'  # Directory to save frames
FRAME_DATA_PATH = 'frame_data.json'  # JSON file to save frame data


# Initialize box annotator for drawing bounding boxes
box_annotator = sv.BoxAnnotator(
    thickness=4,
    text_thickness=4,
    text_scale=2
)

# Open video info and frame generator
video_info = sv.VideoInfo.from_video_path(SOURCE_VIDEO_PATH)
generator = sv.video.get_video_frames_generator(SOURCE_VIDEO_PATH)

# Initialize sequential ID mapping
id_counter = 1
id_map = {}  # Maps tracker_id to a sequential ID
frame_data_list = []  # To store frame data

# Create directory for saving frames
os.makedirs(FRAME_SAVE_DIR, exist_ok=True)

# Define the crossing line position (vertical line for simplicity, adjust as needed)
line_y_position = video_info.height // 2  # Horizontal line in the middle of the frame

# Variables to track vehicle crossings
vehicle_crossings = {'entered': 0, 'exited': 0}
crossing_tracker = {}  # Tracks if a vehicle has crossed the line

# Open output video stream
with sv.VideoSink(TARGET_VIDEO_PATH, video_info) as sink:
    # Iterate through each frame in the video and track objects
    for frame_number, result in enumerate(
        YOLO('yolov8s.pt').track(
            source=SOURCE_VIDEO_PATH,
            tracker='bytetrack.yaml',
            show=False,
            stream=True,
            agnostic_nms=True,
            persist=True
        )
    ):
        # Extract frame and detections
        frame = result.orig_img
        detections = sv.Detections.from_yolov8(result)

        # Handle object IDs (tracker IDs)
        if result.boxes.id is not None:
            for tracker_id in result.boxes.id.cpu().numpy().astype(int):
                # Assign sequential ID if not already assigned
                if tracker_id not in id_map:
                    id_map[tracker_id] = id_counter
                    id_counter += 1

            # Update detections with new sequential IDs
            detections.tracker_id = [id_map[tracker_id] for tracker_id in result.boxes.id.cpu().numpy().astype(int)]

        # Define labels for each detection
        labels = [
            f"ID: {tracker_id} | {model.model.names[class_id]} {confidence:0.2f}"
            for bbox, confidence, class_id, tracker_id in detections
        ]

        # Annotate the frame
        frame = box_annotator.annotate(scene=frame, detections=detections, labels=labels)

        # Check for vehicles crossing the line
        for bbox, _, _, tracker_id in detections:
            center_y = int((bbox[1] + bbox[3]) / 2)  # Get the center y position of the bounding box

            # If vehicle is detected for the first time, initialize it in the tracker
            if tracker_id not in crossing_tracker:
                crossing_tracker[tracker_id] = {'crossed': False, 'last_position': center_y}

            # Check if vehicle is crossing the line
            if not crossing_tracker[tracker_id]['crossed']:
                # Vehicle is crossing the line from below
                if center_y > line_y_position and crossing_tracker[tracker_id]['last_position'] <= line_y_position:
                    vehicle_crossings['entered'] += 1
                    crossing_tracker[tracker_id]['crossed'] = True  # Mark as crossed
                # Vehicle is crossing the line from above
                elif center_y < line_y_position and crossing_tracker[tracker_id]['last_position'] >= line_y_position:
                    vehicle_crossings['exited'] += 1
                    crossing_tracker[tracker_id]['crossed'] = True  # Mark as crossed

            # Update the vehicle's last position
            crossing_tracker[tracker_id]['last_position'] = center_y

        # Draw the crossing line on the frame
        cv2.line(frame, (0, line_y_position), (video_info.width, line_y_position), (0, 255, 0), 2)

        # Add entry and exit counts to the frame
        cv2.putText(frame, f"Entered: {vehicle_crossings['entered']}", (30, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
        cv2.putText(frame, f"Exited: {vehicle_crossings['exited']}", (30, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        # Save current frame to disk
        frame_path = os.path.join(FRAME_SAVE_DIR, f"frame_{frame_number:04d}.jpg")
        cv2.imwrite(frame_path, cv2.cvtColor(frame, cv2.COLOR_RGB2BGR))  # Convert RGB to BGR for OpenCV

        # Collect frame data for JSON
        frame_data = {
            "frame_number": frame_number,
            "detections": [
                {
                    "tracker_id": int(tracker_id),  # Convert to Python int
                    "class_id": int(class_id),      # Convert to Python int
                    "class_name": model.names[class_id],
                    "confidence": float(confidence),  # Convert to Python float
                    "bbox": [float(coord) for coord in bbox]  # Convert bbox to list of floats
                }
                for bbox, confidence, class_id, tracker_id in detections
            ]
        }
        frame_data_list.append(frame_data)

        # Write annotated frame to the output video
        sink.write_frame(frame)

    print(f"Vehicle tracking completed. Entered: {vehicle_crossings['entered']}, Exited: {vehicle_crossings['exited']}")

# Save frame data to a JSON file
with open(FRAME_DATA_PATH, 'w') as json_file:
    json.dump(frame_data_list, json_file, indent=4)

print(f"Frames saved to '{FRAME_SAVE_DIR}' and frame data saved to '{FRAME_DATA_PATH}'.")



video 1/1 (frame 1/538) /kaggle/working/vehicle-counting.mp4: 384x640 4 cars, 1 truck, 9.2ms
video 1/1 (frame 2/538) /kaggle/working/vehicle-counting.mp4: 384x640 4 cars, 1 truck, 8.7ms
video 1/1 (frame 3/538) /kaggle/working/vehicle-counting.mp4: 384x640 4 cars, 1 truck, 8.4ms
video 1/1 (frame 4/538) /kaggle/working/vehicle-counting.mp4: 384x640 3 cars, 1 truck, 8.3ms
video 1/1 (frame 5/538) /kaggle/working/vehicle-counting.mp4: 384x640 3 cars, 1 bus, 8.3ms
video 1/1 (frame 6/538) /kaggle/working/vehicle-counting.mp4: 384x640 3 cars, 1 bus, 8.3ms
video 1/1 (frame 7/538) /kaggle/working/vehicle-counting.mp4: 384x640 3 cars, 1 truck, 8.2ms
video 1/1 (frame 8/538) /kaggle/working/vehicle-counting.mp4: 384x640 2 cars, 1 truck, 8.3ms
video 1/1 (frame 9/538) /kaggle/working/vehicle-counting.mp4: 384x640 3 cars, 1 truck, 8.4ms
video 1/1 (frame 10/538) /kaggle/working/vehicle-counting.mp4: 384x640 3 cars, 1 train, 1 truck, 8.3ms
video 1/1 (frame 11/538) /kaggle/working/vehicle-counting.mp4: 

In [42]:
from IPython.display import FileLink
FileLink(r'output_video.mp4')

/kaggle/working/output_video.mp4